In [1]:
from cProfile import label
from statistics import mode
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout, Flatten


In [2]:
from dataprep import *

In [3]:
a=DataSampling()
a.samplingDF
X_train,X_test,y_train,y_test = a.SampleValidSequences(num_clients=3, test_split=0.3)


In [4]:
print (len(X_train),len(X_test),len(y_train),len(y_test))


14772 8364 14772 8364


Next -- build out table of metrics and the runner
Write the metrics to a CSV for storage
Use an init for the first metric build

Run a few test runs to see what else we need to record.



In [12]:
training_df.groupby('PtID').count()

,RecID,ParentCITYDeviceUploadsID,DeviceDtTm,Units,SortOrd,ValueMMOL,DDate,hourOfDay,series,series_id
PtID,,,,,,,,,,
54,9068,9068,9068,9068,9068,9068,9068,9068,9068,9068
85,2610,2610,2610,2610,2610,2610,2610,2610,2610,2610
86,1834,1834,1834,1834,1834,1834,1834,1834,1834,1834
104,9257,9257,9257,9257,9257,9257,9257,9257,9257,9257
111,3378,3378,3378,3378,3378,3378,3378,3378,3378,3378
159,3824,3824,3824,3824,3824,3824,3824,3824,3824,3824
164,3523,3523,3523,3523,3523,3523,3523,3523,3523,3523
172,3406,3406,3406,3406,3406,3406,3406,3406,3406,3406


In [15]:
ct_df

,RecID,ParentCITYDeviceUploadsID,DeviceDtTm,Units,SortOrd,ValueMMOL,DDate,hourOfDay,series,series_id
PtID,,,,,,,,,,
1,21277,21277,21277,21277,21277,21277,21277,21277,21277,21277
2,3898,3898,3898,3898,3898,3898,3898,3898,3898,3898
4,50122,50122,50122,50122,50122,50122,50122,50122,50122,50122
5,28722,28722,28722,28722,28722,28722,28722,28722,28722,28722
6,4500,4500,4500,4500,4500,4500,4500,4500,4500,4500
...,...,...,...,...,...,...,...,...,...,...
180,4684,4684,4684,4684,4684,4684,4684,4684,4684,4684
181,3181,3181,3181,3181,3181,3181,3181,3181,3181,3181
182,36165,36165,36165,36165,36165,36165,36165,36165,36165,36165


Experiment Plans

Build out models to compare performance on
Look at hyperparameter tuning
ONce models are adequately baked, then move to A and B below

A Experiment on using different patients inputs and keeping track of metrics
B Experiment on using feature engineering and build out metrics further


Implement Data cleaning from development to the data helper functions
Add in ability to look at a per patient basis
Perhaps start with overall data size, and then with a % set to train v test


In [ ]:

def root_mean_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.math.squared_difference(y_true, y_pred)))


class RunModel:

    def __init__(self,X_train, X_test, y_train, y_test):
        #self.X_train = (tf.convert_to_tensor(X_train) - tf.math.reduce_min(tf.convert_to_tensor(X_train))) / (tf.math.reduce_max(tf.convert_to_tensor(X_train)) - tf.math.reduce_min(tf.convert_to_tensor(X_train)))
        #self.y_train = (tf.convert_to_tensor(y_train) - tf.math.reduce_min(tf.convert_to_tensor(y_train))) / (tf.math.reduce_max(tf.convert_to_tensor(y_train)) - tf.math.reduce_min(tf.convert_to_tensor(y_train)))
        #self.X_test = (tf.convert_to_tensor(X_test) - tf.math.reduce_min(tf.convert_to_tensor(X_test))) / (tf.math.reduce_max(tf.convert_to_tensor(X_test)) - tf.math.reduce_min(tf.convert_to_tensor(X_test)))
        #self.y_test = (tf.convert_to_tensor(y_test) - tf.math.reduce_min(tf.convert_to_tensor(y_test))) / (tf.math.reduce_max(tf.convert_to_tensor(y_test)) - tf.math.reduce_min(tf.convert_to_tensor(y_test)))
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)
        self.X_test = np.array(X_test)
        self.y_test = np.array(y_test)
        self.metric_df = pd.DataFrame()

    def rnn_model(self):
        model_rnn = Sequential()
        model_rnn.add(SimpleRNN(50, activation='relu', input_shape=(8,1)))
        model_rnn.add(Dense(10))
        model_rnn.add(Dense(1))
        print('\nRunning RNN model...')
        model_rnn.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
        hist = model_rnn.fit(self.X_train, self.y_train, epochs=7, validation_split=0.2, batch_size=100)
        
        train_loss, train_rmse = model_rnn.evaluate(self.X_train, self.y_train)
        print(f'RNN Model: \nTraining set has a loss (MSE) of {train_loss} with RMSE metric of {train_rmse}')

        test_loss, test_rmse = model_rnn.evaluate(self.X_test, self.y_test)
        print(f'Test set has a loss (MSE) of {test_loss} with RMSE metric of {test_rmse}\n')
        y_pred = model_rnn.predict(self.X_test)
        plt.plot(range(len(y_pred)),y_pred, label='Prediction')
        plt.plot(self.y_test, label='Actual')
        plt.xlabel('Time Series')
        plt.ylabel('Readings')
        plt.title('Simple RNN MODEL')
        plt.legend()
        plt.savefig('Model Fig - SimpleRNN.png')
        plt.clf()
        model_rnn.summary()
        self.metric_df['RNN RMSE'] = hist.history['val_root_mean_squared_error']



    def lstm_model(self):
        model_lstm = Sequential()
        model_lstm.add(LSTM(128, activation='relu', input_shape=(8, 1),return_sequences=True))
        model_lstm.add(Dropout(0.3))
        model_lstm.add(LSTM(64, activation='relu'))
        model_lstm.add(Dropout(0.3))
        model_lstm.add(Dense(20, activation='relu'))
        model_lstm.add(Dense(10, activation='relu'))
        model_lstm.add(Dense(1))
        print('\nRunning the LSTM model...')
        model_lstm.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
        hist = model_lstm.fit(self.X_train, self.y_train, epochs=7, validation_split=0.2, batch_size=100)
        
        train_loss, train_rmse = model_lstm.evaluate(self.X_train, self.y_train)
        print(f'LSTM Model: \nTraining set has a loss (MSE) of {train_loss} with RMSE metric of {train_rmse}')

        test_loss, test_rmse = model_lstm.evaluate(self.X_test, self.y_test)
        print(f'Test set has a loss (MSE) of {test_loss} with RMSE metric of {test_rmse}\n')
        y_pred = model_lstm.predict(self.X_test)
        plt.plot(range(len(y_pred)),y_pred, label='Prediction')
        plt.plot(self.y_test, label='Actual')
        plt.xlabel('Time Series')
        plt.ylabel('Readings')
        plt.title('LSTM MODEL')
        plt.legend()
        plt.savefig('Model Fig - LSTM.png')
        plt.clf()
        model_lstm.summary()
        self.metric_df['LSTM RMSE'] = hist.history['val_root_mean_squared_error']


    def gru_model(self):
        model_gru = Sequential()
        model_gru.add(GRU(50, activation='relu', input_shape=(8,1)))
        model_gru.add(Dense(1))
        print('\nRunning GRU model...')
        model_gru.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
        hist = model_gru.fit(self.X_train, self.y_train, epochs=7, validation_split=0.2, batch_size=100)
        
        train_loss, train_rmse = model_gru.evaluate(self.X_train, self.y_train)
        print(f'GRU Model: \nTraining set has a loss (MSE) of {train_loss} with RMSE metric of {train_rmse}')

        test_loss, test_rmse = model_gru.evaluate(self.X_test, self.y_test)
        print(f'Test set has a loss (MSE) of {test_loss} with RMSE metric of {test_rmse}\n')
        y_pred = model_gru.predict(self.X_test)
        plt.plot(range(len(y_pred)),y_pred, label='Prediction')
        plt.plot(self.y_test, label='Actual')
        plt.xlabel('Time Series')
        plt.ylabel('Readings')
        plt.title('GRU MODEL')
        plt.legend()
        plt.savefig('Model Fig - GRU.png')
        plt.clf()
        model_gru.summary()
        self.metric_df['GRU RMSE'] = hist.history['val_root_mean_squared_error']


    def cnn_lstm_model(self):
        model_cnn_lstm = Sequential()
        model_cnn_lstm.add(tf.keras.layers.Conv1D(32, 2, activation='relu', input_shape=(8,1)))
        model_cnn_lstm.add(tf.keras.layers.MaxPooling1D((1)))
        model_cnn_lstm.add(LSTM(10, activation='relu', return_sequences=True))
        model_cnn_lstm.add(Flatten())
        model_cnn_lstm.add(Dense(1))
        print('\nRunning the CNN+LSTM model...')
        model_cnn_lstm.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
        hist = model_cnn_lstm.fit(self.X_train, self.y_train, epochs=7, validation_split=0.2, batch_size=100)
        
        train_loss, train_rmse = model_cnn_lstm.evaluate(self.X_train, self.y_train)
        print(f'CNN+LSTM Model: \nTraining set has a loss (MSE) of {train_loss} with RMSE metric of {train_rmse}')

        test_loss, test_rmse = model_cnn_lstm.evaluate(self.X_test, self.y_test)
        print(f'Test set has a loss (MSE) of {test_loss} with RMSE metric of {test_rmse}\n')
        y_pred = model_cnn_lstm.predict(self.X_test)
        print(y_pred.shape)
        plt.plot(range(len(y_pred)),y_pred, label='Prediction')
        plt.plot(self.y_test, label='Actual')
        plt.xlabel('Time Series')
        plt.ylabel('Readings')
        plt.title('CNN+LSTM MODEL')
        plt.legend()
        plt.savefig('Model Fig - CNN+LSTM.png')
        plt.clf()
        model_cnn_lstm.summary()
        self.metric_df['CNN+LSTM RMSE'] = hist.history['val_root_mean_squared_error']


    def main(self):
        self.rnn_model()
        self.lstm_model()
        self.gru_model()
        self.cnn_lstm_model()
        self.metric_df.plot(xlabel='Epochs', ylabel='RMSE', legend=True)
        plt.show()


In [6]:


def seriesToTimeSeries(self, X, step_length=8,forecast_dist=6):
    y=[]
    reshapedX = []
    for i in range(len(X)-forecast_dist-step_length):
        y.append(X[i+step_length+forecast_dist])
        reshapedX.append(X[i:i+step_length])
    return reshapedX,y

def shapeSeriesFromDF(df,indexForSelection):
    
    an_X = df[df['series_id']==indexForSelection[0]].ValueMMOL.tolist()
    an_X, y = seriesToTimeSeries(an_X)
    X_=an_X
    y_=y

    for i in indexForSelection[1:]:
        an_X = df[df['series_id']==i].ValueMMOL.tolist()
        an_X, y = seriesToTimeSeries(an_X)
        
        X_ = X_+an_X
        y_ = y_+y
    return X_,y_



def SampleValidSequences(self, num_clients=8, test_split=0.3,seed=1):
    samplingDF = self.samplingDF
    ## cleaning up the data -- Resetting data types
    
    random.seed(seed)
    
    new_df = samplingDF.groupby('series_id').count()
    ct_df = samplingDF.groupby('PtID').count()

    client_list = ct_df.index.to_numpy()
    cl_ind = client_list[random.sample(range(0,len(client_list)),num_clients)] ##clientids to use for the training

    cl_df = samplingDF[samplingDF.PtID.isin(cl_ind)] ## list of all samples relative to these clients

    series_select = cl_df.groupby('series_id').count()
    series_select = series_select.sample(frac=1)
    series_select = series_select.index.to_list()

    index_cut = int((1-test_split) * len(series_select))
    train_index = series_select[0:index_cut]
    test_index=series_select[index_cut:]

    training_df = samplingDF[samplingDF.series_id.isin(train_index)]
    testing_df = samplingDF[samplingDF.series_id.isin(test_index)]

    ## build training dataset
    X_train,y_train = shapeSeriesFromDF(training_df,train_index)
    X_test,y_test = shapeSeriesFromDF(testing_df,test_index)

    return X_train, X_test, y_train, y_test



In [8]:
x,y=shapeSeriesFromDF(a.samplingDF,[1,3,45])  

TypeError: seriesToTimeSeries() missing 1 required positional argument: 'X'